## Obtaining the IMDb movie review dataset

In [8]:
import pyprind
import pandas as pd
import os

In [2]:
pbar = pyprind.ProgBar(50000)

0%                          100%
[                              ]

In [4]:
# create dataframe and labels

labels = {'pos':1, 'neg':0}

df = pd.DataFrame()

In [9]:
# change the directory

os.chdir('/Users/Bya/Dropbox/Research/datas/')

In [8]:
# Read all text file, and append to dataframe
# it will take 10 minutes

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
            

[##############################] | ETA[sec]: 0.000 
Total time elapsed: 1254.487 sec


In [9]:
df.columns = ['reviews', 'sentiment']

In [ ]:
# Save dataframe as CSV

import numpy as np
np.random.seed(0)

df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False)

## 0. Read the `movie_data` as `dataframe`

In [25]:
# read saved CSV file
import pandas as pd
import os

os.chdir('/Users/Bya/Dropbox/Research/datas/')

df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


# 1. bag-of-words model
## Transforming words into feature vectors

In [1]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# 1 gram(unigram). ngram_range=(2,2) is 2 gram

count = CountVectorizer()

In [3]:
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'
    ])

In [4]:
# fit_transform construct bag-of-words model
# and transform into sparse feature vectors

bag = count.fit_transform(docs)

In [5]:
# the vocabulary is stored in a Python dictionary, 
# which maps the unique words that are mapped to integer indices

print(count.vocabulary_)

{'sweet': 4, 'the': 5, 'weather': 6, 'and': 0, 'is': 1, 'sun': 3, 'shining': 2}


In [6]:
# feature vectors

print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


# 2. tf-idfs
## Assessing word relevancy via term frequency-inverse document frequency

* **`tf(t,d)`**—the number of times a term **`t`** occurs in a document **`d`**


* **`df(d,t)`**—is the number of documents **`d`** that contain the term **`t`** 


* **`idf(d,t)`**:
    
    **`idf(t,d) = log(n_d / (1 + df(d,t)))`**


* **term frequency** and the **inverse document frequency**:
    
    **`tf-idf(t,d) = tf(t,d) × idf(t,d)`**

* **`tf(t,d)`**— **`t`** がドキュメント**`d`**に何回あるか


* **`df(d,t)`**— **`t`**を含む**`d`**がどのぐらいあるか

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
tfidf = TfidfTransformer()

np.set_printoptions(precision=2)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.56  0.56  0.    0.43  0.  ]
 [ 0.    0.43  0.    0.    0.56  0.43  0.56]
 [ 0.4   0.48  0.31  0.31  0.31  0.48  0.31]]


# 3. Cleaning text data

In [26]:
# last 50 characters of first review

df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

### regex cleaner

In [20]:
import re

def preprocessor(text):
    # remove HTML
    text = re.sub('<[^>]*>', '', text)
    
    # find emotiocons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # remove all non-word, lower_case, add emoticons, remove nose
    text = re.sub('[\W]+', ' ', text.lower()) + \
                ''.join(emoticons).replace('-', '')
    return text

In [27]:
# tests
print(preprocessor(df.loc[0, 'review'][-50:]))

print(preprocessor("</a>This :) is :( a test :-)!"))

is seven title brazil not available
this is a test :):(:)


Apply **`preprocessor`** function to all movie reviews in **`DataFrame`**

In [28]:
df['review'] = df['review'].apply(preprocessor)

### Preprocessing documents into tokens

In [3]:
# Simple Tokenizer

def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']


['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [4]:
# Tokenizer with Porter Stemmer

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

### Stop-words

In [42]:
# set of 127 English stop-words
from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 4. Training a logistic regression model for document classification

In [43]:
# divide the DataFrame of cleaned text documents 
# into 25,000 documents for training and 25,000 documents for testing

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values

X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [44]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

In [48]:
param_grid = [
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer,
                           tokenizer_porter],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0]
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer,
                           tokenizer_porter],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0]
    }
]

In [49]:
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(random_state=0))])

In [51]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                          scoring='accuracy',
                          cv=5, verbose=1,
                          n_jobs=-1)

In [52]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done 200 jobs       | elapsed: 164.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 251.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ovr',
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid=[{'clf__C': [1.0, 10.0, 100.0], 'vect__tokenizer': [<function tokenizer at 0x10e775ae8>, <function tokenizer_porter at 0x10e775d08>], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'h...ry', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'], None], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, score_func=No

### best parameter set

In [53]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'vect__tokenizer': <function tokenizer at 0x10e775ae8>, 'vect__stop_words': None, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1)} 


In [54]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

CV Accuracy: 0.897


In [55]:
clf = gs_lr_tfidf.best_estimator_

In [56]:
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.898


# 5. Working with bigger data - online algorithms and out-of-core learning

In [5]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [10]:
# generator function, 
# reads in and returns one document at a time
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [11]:
next(stream_docs(path='./movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [12]:
# will take a document stream from the stream_docs function
# return a particular number of documents specified by the size parameter
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [13]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

### out-of-core learning

In [14]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 34.023 sec


In [15]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [16]:
clf = clf.partial_fit(X_test, y_test)

In [17]:
clf.score

<bound method SGDClassifier.score of SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)>

In [18]:
clf.intercept_

array([ 0.08371193])

In [22]:
clf.coef_

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

<bound method SGDClassifier.get_params of SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)>

## Save the training model

In [79]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop,
            open(os.path.join(dest, 'stopwords.pkl'),'wb'),
            protocol=4)

pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4)